# Try Reproducing Everything to Stock

In [54]:
import sys
import os
sys.path.append(os.path.abspath("D:\\Python\\commodity_futures_price"))

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

# %load_ext autoreload
# %autoreload 2
%reload_ext autoreload
from model.savvy_sh import SavvySh

import yfinance as yf

from model.factor import Factor

### Download Necessary Data From yfinance (if haven't downloaded)

In [55]:
ticker = "AAPL"
# download if not found in ../../data/yfinance/
if not os.path.exists(f"../../data/yfinance/{ticker}.csv"):
    data = yf.Ticker(ticker)
    data = data.history(period="15Y", repair=True)
    data.index = data.index.date
    data.to_csv(f"../../data/yfinance/{ticker}.csv")
data = pd.read_csv(f"../../data/yfinance/{ticker}.csv", index_col=0, parse_dates=True)
# check if there is any row with Repair = True
data[data['Repaired?'] == True]

factor = Factor(data, ticker=ticker)

### Momentum

In [56]:
factor.add_momentum()

### Skewness

In [57]:
factor.add_skewness()

### Volatility

In [58]:
factor.add_Volatility()

### Open Interest

In [59]:
factor.add_Open_Interest()

### Value

In [60]:
factor.add_Value()

In [61]:
data.to_csv(f'../../data/processed_stock/cleaned_{ticker}.csv')